In [1]:
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Load Data

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, y_train, X_test, y_test = X_train.astype(float), y_train.astype(float), X_test.astype(float), y_test.astype(float)
X_train = X_train.reshape(-1,28 * 28) / 255
X_test = X_test.reshape(-1,28 * 28) / 255

print("TRAIN DATA : ", X_train.shape, y_train.shape)
print("TEST  DATA : ", X_test.shape, y_test.shape)

TRAIN DATA :  (60000, 784) (60000,)
TEST  DATA :  (10000, 784) (10000,)


In [3]:
sample_data = X_train[:64]

# Modeling

## Variational Encoder

### FC Block

In [4]:
class FC_Block(tf.keras.layers.Layer) : 
    def __init__(self, units = 32) : 
        super(FC_Block, self).__init__()
        self.units = units
        self.linear = tf.keras.layers.Dense(units, kernel_initializer = 'he_normal')
        self.batch = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training = False) : 
        x = self.linear(inputs)
        x = self.batch(x, training = training) 
        x = self.relu(x)
        x = self.dropout(x)

        return x

In [5]:
class Sampling(tf.keras.layers.Layer) : 

    def call(self, inputs) : 
        mu, sigma = inputs
        eps = tf.random.normal(shape = tf.shape(sigma))
        return mu + tf.exp(0.5 * sigma) * eps

In [6]:
class Encoder(tf.keras.layers.Layer) : 
    def __init__(self, block, sample) : 
        super(Encoder, self).__init__()
        self.sample = sample
        self.fc1 = block(256)
        self.fc2 = block(64)
        self.mu = tf.keras.layers.Dense(10)
        self.sigma = tf.keras.layers.Dense(10)

    def call(self, inputs) : 
        x = self.fc1(inputs)
        x = self.fc2(x)
        mu = self.mu(x)
        sigma = self.sigma(x)
        z = self.sample([mu,sigma])
        return mu, sigma, z

In [7]:
encoder = Encoder(FC_Block, Sampling)

In [8]:
class Decoder(tf.keras.layers.Layer) : 
    def __init__(self, block) : 
        super(Decoder ,self).__init__()
        self.fc1 = block(64)
        self.fc2 = block(256)
        self.linear = tf.keras.layers.Dense(28*28)
        
    def call(self, inputs) : 
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.linear(x)
        return tf.sigmoid(x)

In [17]:
decoder = Decoder(FC_Block)

In [ ]:
class VAE(tf.keras.Model) : 
    def __init__(self, encoder, decoder) :
        